In [1]:
import tensorflow as tf
import os
from operator import add

from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import mnist
import numpy as np

import pickle
import timeit

from tensorflow import keras # Required for Tensorboard
from datetime import datetime # Required for Tensorboard
from copy import deepcopy

import time

In [2]:
def LeNet():
    Convolution2D = tf.keras.layers.Convolution2D
    MaxPooling2D = tf.keras.layers.MaxPooling2D
    Flatten = tf.keras.layers.Flatten
    Dense = tf.keras.layers.Dense

    model = tf.keras.models.Sequential()

    with tf.name_scope("LeNet"):
        with tf.name_scope("Convolution_Block"):
            # Add the first convolution layer
            model.add(Convolution2D(
                filters = 20,
                kernel_size = (5, 5),
                padding = "same",
                input_shape = (28, 28, 1),
                activation="relu",
                name="Conv1"))

            # Add a pooling layer
            model.add(MaxPooling2D(
                pool_size = (2, 2),
                strides =  (2, 2),
                name="MaxPool1"))

            # Add the second convolution layer
            model.add(Convolution2D(
                filters = 50,
                kernel_size = (5, 5),
                padding = "same",
                activation="relu",
                name="Conv2"))

            # Add a second pooling layer
            model.add(MaxPooling2D(
                pool_size = (2, 2),
                strides = (2, 2),
                name="MaxPool2"))

        # Flatten the network
        model.add(Flatten())

        with tf.name_scope("Dense_Block"):
            # Add a fully-connected hidden layer
            model.add(Dense(500,
                activation="relu",
                name="Dense3"))

            # Add a fully-connected output layer
            model.add(Dense(10,
                activation="softmax",
                name="Dense4"))
    return model

In [ ]:
if MOD_WEIGHTS == True:
    k = len(layers)
    for i in range(0, k-1, 1):
        fact = 1.0 + ((layers[i].W.shape[0]*layers[i].W.shape[1])/
        (layers[i+1].W.shape[0]*layers[i+1].W.shape[1]))
        layers[k-1-i].W*=fact
        print(fact)


In [3]:
model = LeNet()

# Instantiate an optimizer to train the model.
optimizer = keras.optimizers.SGD(learning_rate=1e-3)

# Compile the network
model.compile(
    loss="categorical_crossentropy",
    optimizer=optimizer,
    metrics=["accuracy"],
    run_eagerly=False
)

AUTOTRAIN = True
if AUTOTRAIN:
    
    dataset = mnist.load_data()

    train_data = dataset[0][0]
    train_labels = dataset[0][1]
    test_data = dataset[1][0]
    test_labels = dataset[1][1]

    # Reshape the data to a (70000, 28, 28, 1) tensord
    train_data = train_data.reshape([*train_data.shape,1]) / 255.0
    test_data = test_data.reshape([*test_data.shape,1]) / 255.0
    train_labels = np.eye(10)[train_labels]

    # Tranform test labels to one-hot encoding
    test_labels = np.eye(10)[test_labels]

    hist = model.fit(
            train_data,
            train_labels,
            batch_size=128,
            epochs=1,
            validation_split=0.33,
            verbose=1,
            callbacks=[])
else:
    # TODO:
    # https://keras.io/guides/writing_a_training_loop_from_scratch/
    # https://keras.io/api/losses/probabilistic_losses/#categoricalcrossentropy-class
    batch_size = 128
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
    x_train = x_train.reshape([*x_train.shape,1]) / 255.0 ##np.reshape(x_train, (-1, 784)) 
    x_test = x_test.reshape([*x_test.shape,1]) / 255.0
    
    train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)
    
    x_val = x_train[-1000:]
    y_val = y_train[-1000:]
    x_train = x_train[:-100000]
    y_train = y_train[:-100000]
    val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
    val_dataset = val_dataset.batch(128)
    
    # Instantiate a loss function.
    loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    
    # Prepare the metrics.
    train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
    val_acc_metric = keras.metrics.SparseCategoricalAccuracy()
    
    @tf.function
    def train_step(x,y):
        # Open a GradientTape to record the operations run
        # during the forward pass, which enables autodifferentiation.
        with tf.GradientTape() as tape:

            # Run the forward pass of the layer.
            # The operations that the layer applies
            # to its inputs are going to be recorded
            # on the GradientTape.
            logits = model(x_batch_train, training=True)  # Logits for this minibatch

            # Compute the loss value for this minibatch.
            loss_value = loss_fn(y_batch_train, logits)

        # Use the gradient tape to automatically retrieve
        # the gradients of the trainable variables with respect to the loss.
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        train_acc_metric.update_state(y_batch_train, logits)
        return loss_value
    
    @tf.function
    def test_step(x, y):
        val_logits = model(x, training=False)
        # Update val metrics
        val_acc_metric.update_state(y, val_logits)


    epochs = 3
    for epoch in range(epochs):
        print("\nStart of epoch %d" % (epoch,))
        start_time = time.time()

        # Iterate over the batches of the dataset.
        for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):

            
            loss_value = train_step(x_batch_train, y_batch_train)
            
            # Log every 200 batches.
            if step % 200 == 0:
                print(
                    "Training loss (for one batch) at step %d: %.4f"
                    % (step, float(loss_value))
                )
                print("Seen so far: %s samples" % ((step + 1) * batch_size))

        # Display metrics at the end of each epoch.
        train_acc = train_acc_metric.result()
        print("Training acc over epoch: %.4f" % (float(train_acc),))

        # Reset training metrics at the end of each epoch
        train_acc_metric.reset_states()

        # Run a validation loop at the end of each epoch.
        for x_batch_val, y_batch_val in val_dataset:
            test_step(x_batch_val, y_batch_val)
        val_acc = val_acc_metric.result()
        val_acc_metric.reset_states()
        print("Validation acc: %.4f" % (float(val_acc),))
        print("Time taken: %.2fs" % (time.time() - start_time))

315/315 [==============================] - 115s 364ms/step - loss: 2.2791 - accuracy: 0.2540 - val_loss: 2.2588 - val_accuracy: 0.3472


In [25]:
test_labels = np.eye(10)[y_test]
(loss, accuracy) = model.evaluate(
            x_test,
            test_labels,
            batch_size=128,
            verbose=1)

79/79 [==============================] - 6s 79ms/step - loss: 2.3019 - accuracy: 0.1060


In [ ]:
#lenet.layers
inv_layers = model.layers#.reverse()
inv_layers.reverse()
for i, layer in enumerate(model.layers):
    print(layer.name)
    try:
        cce = tf.keras.losses.CategoricalCrossentropy()
        model.layers[i].set_weights([layer.get_weights()[0]*1.0, layer.get_weights()[1]])
    except Exception as e:
        print('Failed for', layer.name)
        print(e)
        
print(model.layers[5].get_weights()[0])